This is a simplified version of the [CountBot](https://www.kaggle.com/superant/rps-geometry-silver-rank-by-minimal-logic) without complex numbers. I removed the complex number for if you plan to leave all the steps discrete.

It could serve as a baseline to test if you have found a meaningful improvement upon the core initial step of a history matcher bot.

In [ ]:
%%writefile basiccount.py
import operator
import numpy as np
from typing import List
from collections import namedtuple
import traceback
import sys


HistMatchResult = namedtuple("HistMatchResult", "idx length")


def find_all_longest(seq, max_len=None) -> List[HistMatchResult]:
    """
    Find all indices where end of `seq` matches some past.
    """
    result = []

    i_search_start = len(seq) - 2

    while i_search_start > 0:
        i_sub = -1
        i_search = i_search_start
        length = 0

        while i_search >= 0 and seq[i_sub] == seq[i_search]:
            length += 1
            i_sub -= 1
            i_search -= 1

            if max_len is not None and length > max_len:
                break

        if length > 0:
            result.append(HistMatchResult(i_search_start + 1, length))

        i_search_start -= 1

    result = sorted(result, key=operator.attrgetter("length"), reverse=True)

    return result


class Pred:
    def __init__(self, *, alpha):
        self.offset_probs = np.full(fill_value=1/3, shape=3)
        self.alpha = alpha
        self.last_feat = None

    def train(self, target):
        if self.last_feat is not None:
            offset = (target - self.last_feat) % 3

            self.offset_probs *= (1 - self.alpha)  # technically you should better decay towards the mean instead?
            self.offset_probs[offset] += self.alpha

    def predict(self, feat):
        result = np.roll(self.offset_probs, shift=feat)
        self.last_feat = feat
        return result
    
    
class BaseAgent:
    def __init__(self):
        self.my_hist = []
        self.opp_hist = []
        self.my_opp_hist = []
        self.outcome_hist = []
        self.step = None

    def __call__(self, obs, conf):
        try:
            if obs.step == 0:
                action = np.random.choice(3)
                self.my_hist.append(action)
                return action

            self.step = obs.step

            opp = int(obs.lastOpponentAction)
            my = self.my_hist[-1]

            self.my_opp_hist.append((my, opp))
            self.opp_hist.append(opp)

            outcome = {0: 0, 1: 1, 2: -1}[(my - opp) % 3]
            self.outcome_hist.append(outcome)

            action = self.action()

            self.my_hist.append(action)

            return action
        except Exception:
            traceback.print_exc(file=sys.stderr)
            raise

    def action(self):
        """
        This has to be implemented to return the action
        """
        pass


class Agent(BaseAgent):
    def __init__(self, alpha=0.01):
        super().__init__()
        self.predictor = Pred(alpha=alpha)

    def action(self):
        self.train()
        probs = self.preds()
        return np.random.choice(3, p=probs)

    def train(self):
        last_beat_opp = (self.opp_hist[-1] + 1) % 3
        self.predictor.train(last_beat_opp)

    def preds(self):
        hist_match = find_all_longest(self.my_opp_hist, max_len=20)

        if not hist_match:
             return [1/3,1/3,1/3]

        feat = self.opp_hist[hist_match[0].idx]
        pred = self.predictor.predict(feat)
        return pred
    
    
agent = Agent()


def call_agent(obs, conf):
    return agent(obs, conf)

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("rps")
env.reset()
env.run(["basiccount.py", "statistical"]);
env.render(mode="ipython", width=600, height=450)